In [1]:
import ee
ee.Initialize()

In [3]:
from geetools import tools, utils
import geedatasets

In [4]:
import ipygee as ui

## Get a NDVI Image

In [5]:
l8 = geedatasets.landsat.Landsat8SR()

In [6]:
p = ee.Geometry.Point([-72,-42])

In [7]:
col = l8.collection().filterBounds(p)

In [8]:
i = ee.Image(col.first())

In [9]:
ndvi = l8.ndvi.apply(i)

## Get point grid

In [10]:
points = ndvi.sample(scale=10000, geometries=True)

# Linear Function

In [15]:
d = tools.image.linearFunction(ndvi, 'NDVI', mean=0.3, range_min=0, range_max=1, min=2, max=5)
result = ndvi.addBands(d)

In [17]:
chart = ui.chart.Image.bandsByRegion(result, points.limit(100), xProperty='NDVI')

# Gauss Function

In [18]:
gauss = tools.image.gaussFunction(ndvi, 'NDVI', 0, 1, mean=0.3, output_min=0, output_max=1, stretch=2)
gauss = ndvi.addBands(gauss)

In [19]:
chart_gauss = ui.chart.Image.bandsByRegion(gauss, points.limit(100), xProperty='NDVI', bands=['gauss'])

In [20]:
chart_gauss.renderWidget(width=800)

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

# Normal Distribution

In [22]:
normal = tools.image.normalDistribution(ndvi, 'NDVI', scale=30, maxPixels=1e13)
result_n = ndvi.addBands(normal)

In [23]:
chart_normal = ui.chart.Image.bandsByRegion(result_n, points.limit(100), xProperty='NDVI', bands=['normal_distribution'])

# Charts

In [24]:
charts = chart.cat(chart_gauss, chart_normal)

In [25]:
charts.renderWidget(width=800)

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [26]:
charts.dataframe

,linear_function,gauss,normal_distribution
-0.264706,0.193277,0.005691,0.499408
-0.088890,0.444442,0.088583,0.567368
-0.078514,0.459266,0.100673,0.570398
-0.003532,0.566383,0.228679,0.588462
-0.000902,0.570140,0.234580,0.588968
...,...,...,...
0.797962,0.288626,0.018535,0.368387
0.800728,0.284674,0.017717,0.366860
0.837381,0.232313,0.009458,0.346635
0.864023,0.194253,0.005767,0.331999
